In [1]:
%%javascript
//debugging to see if jupyter-klampt-widget is installed properly
console.log(require.s.contexts._.defined);
var kf = require('nbextensions/klampt/KlamptFrontend');
console.log(kf);

<IPython.core.display.Javascript object>

In [1]:
#Python 2/3 compatibility
from __future__ import print_function,division
import time
from klampt import *
from klampt.math import vectorops,so3,se3
from klampt.vis.ipython import KlamptWidget

world = WorldModel()
world.loadFile("../data/athlete_plane.xml")
kvis = KlamptWidget(world)
#If you'd like to hide the print output, uncomment this line
#clear_output()
display(kvis)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

***  klampt.vis: using Qt5 as the visualization backend  ***


KlamptWidget(scene={'metadata': {'version': 4.4, 'type': 'Object', 'fullscene': True, 'generator': 'Klampt thr…

In [2]:
#add testing

robot = world.robot(0)
q = robot.getConfig()
q[2] += 0.5
q1 = [v for v in q]
q[2] += 0.5
q2 = [v for v in q]
#kvis.add('milestones',[q1,q2])
#kvis.set_color('milestones',0,1,0,0.5)

from klampt.model.trajectory import Trajectory
traj = Trajectory([0,1],[q1,q2])
q1[0] += 0.5
q2[0] += 0.5
kvis.add('milestones',traj)
kvis.setColor('milestones',0,1,0,0.5)

In [4]:
#Testing screenshots
%%javascript
var w = window.open('', '');
w.document.title = "Screenshot";
var img = new Image();
img.src = $(".p-Widget > div > canvas")[0].toDataURL();
w.document.body.appendChild(img);  

<IPython.core.display.Javascript object>